# 1. Tổng quan

```Chain-of-Thought (CoT)``` Prompting là kỹ thuật kích hoạt khả năng suy luận của mô hình ngôn ngữ lớn (LLMs) bằng cách yêu cầu chúng trình bày các bước trung gian trong quá trình giải quyết vấn đề. Thay vì đưa ra câu trả lời trực tiếp, CoT hướng dẫn mô hình "nghĩ thành tiếng" thông qua một chuỗi lập luận logic, tương tự cách con người phân tích vấn đề phức tạp. Kỹ thuật này đặc biệt hiệu quả cho các nhiệm vụ đòi hỏi tính toán đa bước, suy luận dựa trên tri thức hoặc ra quyết định có cấu trúc

# 2. Đặc điểm
* Hướng dẫn suy luận từng bước: CoT buộc mô hình chia nhỏ vấn đề thành các bước logic liên tiếp, giảm thiểu sai sót do xử lý thông tin quá tải.
* Phù hợp với nhiệm vụ phức tạp: Hiệu quả vượt trội trong các tác vụ như giải toán, lập luận thông thường (commonsense reasoning), và xử lý câu hỏi đa tầng (multi-hop QA).
* Tăng tính minh bạch: Quá trình suy luận được hiển thị rõ ràng, giúp người dùng kiểm tra tính hợp lý của kết quả.
* Hiệu quả phụ thuộc vào kích thước mô hình: Mô hình càng lớn (ví dụ: GPT-4, PaLM) càng phát huy tốt CoT, trong khi mô hình nhỏ có thể cho kết quả kém hơn.
* Không cần fine-tuning: Áp dụng trực tiếp thông qua prompt mà không cần điều chỉnh trọng số mô hình.

# 3. Best practice
Để tối ưu hóa CoT Prompting, cần lưu ý:

* Sử dụng câu lệnh kích hoạt: Thêm cụm từ như "Let’s think step by step" hoặc "Hãy suy nghĩ từng bước" vào cuối prompt để kích hoạt chế độ suy luận.
* Kết hợp few-shot examples: Cung cấp 2-3 ví dụ minh họa quy trình suy luận mẫu, đặc biệt với tác vụ chuyên biệt

# 4. Ví dụ

In [1]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()

True

## 4.1. Zero-shot CoT
* Cách dùng: Thêm cụm từ kích hoạt (ví dụ: "Hãy suy nghĩ từng bước") vào prompt.
* Ưu điểm: Không cần ví dụ mẫu.

In [2]:
model = init_chat_model(
    model='gpt-4o-mini', 
    model_provider='openai',
    temperature=0.1
)


messages = [
    SystemMessage('Trong vai trò chuyên gia toán học, hãy giúp tôi giải các bài toán sau'),
    HumanMessage('Giải phương trình x^2 - 5x + 6 = 0. Hãy suy nghĩ từng bước.'),
]

response = model.invoke(messages)
print(response.content) 

Để giải phương trình bậc hai \( x^2 - 5x + 6 = 0 \), chúng ta sẽ sử dụng công thức nghiệm của phương trình bậc hai, có dạng:

\[
x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
\]

Trong phương trình của chúng ta, các hệ số là:
- \( a = 1 \)
- \( b = -5 \)
- \( c = 6 \)

Bước 1: Tính giá trị của \( b^2 - 4ac \).

\[
b^2 = (-5)^2 = 25
\]
\[
4ac = 4 \cdot 1 \cdot 6 = 24
\]
\[
b^2 - 4ac = 25 - 24 = 1
\]

Bước 2: Tính nghiệm của phương trình.

Áp dụng vào công thức nghiệm:

\[
x = \frac{-(-5) \pm \sqrt{1}}{2 \cdot 1}
\]
\[
x = \frac{5 \pm 1}{2}
\]

Bước 3: Tính các giá trị của \( x \).

- Nghiệm thứ nhất:
\[
x_1 = \frac{5 + 1}{2} = \frac{6}{2} = 3
\]

- Nghiệm thứ hai:
\[
x_2 = \frac{5 - 1}{2} = \frac{4}{2} = 2
\]

Vậy, phương trình \( x^2 - 5x + 6 = 0 \) có hai nghiệm:

\[
x_1 = 3 \quad \text{và} \quad x_2 = 2
\]


## 4.2. Few-shot CoT
Cách dùng: Cung cấp các ví dụ mẫu có kèm lập luận.


In [3]:
model = init_chat_model(
    model='gpt-4o-mini', 
    model_provider='openai',
    temperature=0.1
)


messages = [
    SystemMessage("""
Trong vai trò chuyên gia toán học, hãy giúp tôi giải phương trình bằng cách trình bày từng bước như các ví dụ sau:

**Ví dụ 1**:
Phương trình: x² - 5x + 6 = 0
Bước 1: Xác định hệ số a=1, b=-5, c=6
Bước 2: Tính delta Δ = b² - 4ac = (-5)² - 4*1*6 = 25 - 24 = 1
Bước 3: Vì Δ > 0, phương trình có 2 nghiệm phân biệt
Bước 4: x1 = (-b + √Δ)/(2a) = (5 + 1)/2 = 3
Bước 5: x2 = (-b - √Δ)/(2a) = (5 - 1)/2 = 2
→ Kết quả: x=3 hoặc x=2

**Ví dụ 2**:
Phương trình: 2x² + 4x - 6 = 0
Bước 1: Chia cả 2 vế cho 2 → x² + 2x - 3 = 0
Bước 2: Tính Δ = 2² - 4*1*(-3) = 4 + 12 = 16
Bước 3: √Δ = 4
Bước 4: x = [-2 ± 4]/2 → x1=1, x2=-3
→ Kết quả: x=1 hoặc x=-3
"""),
    HumanMessage('x^2 - 5x + 6 = 0'),
]

response = model.invoke(messages)
print(response.content) 

Phương trình: x² - 5x + 6 = 0

Bước 1: Xác định hệ số a=1, b=-5, c=6

Bước 2: Tính delta Δ = b² - 4ac = (-5)² - 4*1*6 = 25 - 24 = 1

Bước 3: Vì Δ > 0, phương trình có 2 nghiệm phân biệt

Bước 4: Tính nghiệm x1 = (-b + √Δ)/(2a) = (5 + 1)/2 = 3

Bước 5: Tính nghiệm x2 = (-b - √Δ)/(2a) = (5 - 1)/2 = 2

→ Kết quả: x=3 hoặc x=2
